In [2]:
import pandas as pd
import numpy as np
import sklearn as sk

import spicy as sp

train_data = pd.read_csv('Credit.csv')


ModuleNotFoundError: No module named 'spicy'

# Denifinition  des matrice de travail
On definit les matrices de travail
* $X\in  Rn^{xp}$ la matrice des co-variables
* $Y\in R_n$ le vecteur des outputs

In [3]:
train_data.head(10)

,Unnamed: 0,ID,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,1,1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,2,2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,3,3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,4,4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,5,5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331
5,6,6,80.180,8047,569,4,77,10,Male,No,No,Caucasian,1151
6,7,7,20.996,3388,259,2,37,12,Female,No,No,African American,203
7,8,8,71.408,7114,512,2,87,9,Male,No,No,Asian,872
8,9,9,15.125,3300,266,5,66,13,Female,No,No,Caucasian,279
9,10,10,71.061,6819,491,3,41,19,Female,Yes,Yes,African American,1350


Pour $X$ on va retirer de data les colonnes Unamed: 0 ID et Blance et pour $Y$ on ne va garder que la colonne Balence.

In [4]:
X = train_data.drop(["Unnamed: 0", "ID", "Balance"], axis=1)

In [13]:
Y = train_data.Balance
Z = pd.get_dummies(X, drop_first=True)
print(Z.head(10))
Z = np.array(Z)
ZZ = np.linalg.inv(np.dot(np.matrix.transpose(Z), Z))
thetahat = np.dot(np.dot(ZZ, np.matrix.transpose(Z)), Y)
thetahat

    Income  Limit  Rating  Cards  Age  Education  Gender_Female  Student_Yes  \
0   14.891   3606     283      2   34         11              0            0   
1  106.025   6645     483      3   82         15              1            1   
2  104.593   7075     514      4   71         11              0            0   
3  148.924   9504     681      3   36         11              1            0   
4   55.882   4897     357      2   68         16              0            0   
5   80.180   8047     569      4   77         10              0            0   
6   20.996   3388     259      2   37         12              1            0   
7   71.408   7114     512      2   87          9              0            0   
8   15.125   3300     266      5   66         13              1            0   
9   71.061   6819     491      3   41         19              1            1   

   Married_Yes  Ethnicity_Asian  Ethnicity_Caucasian  
0            1                0                    1  
1        

array([ -7.35829145e+00,   3.38649632e-01,  -1.26598926e+00,
         1.45654232e+01,  -2.33422514e+00,  -1.45973335e+01,
        -2.86899596e+01,   4.25572993e+02,  -2.31227096e+01,
        -2.73532004e+01,  -3.18577541e+01])

On doit maintenant definir une fonction $f: x\rightarrow f(x) = x'\widehat{\theta}$

In [53]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
fhat = model.fit(X=Z, y=Y)
fhat.coef_
fhat.predict(np.array([[14.3, 2000, 180, 1, 32, 11, 0, 1, 0, 0, 0]]))

array([ 407.33537497])

# Regression polynomiale

In [54]:
poly = sk.preprocessing.PolynomialFeatures(3)
PhiX = poly.fit_transform(Z)
fhat2 = model.fit(X = PhiX, y = Y)

# Comparaison des risques de prediction
On calcule
$$
    \frac1 N\sum({\hat{f}(X_i) = Y_i})^2
$$

In [55]:
#np.mean((fhat.predict(Z) - Y)**2)
np.mean((fhat2.predict(PhiX)-Y)**2)

298.6966042066154

# Validation croisee
On va separer les donnees en un echantillon d'apprentissage et un echantillon test. On va entrainer le predicteur sur l'echantillon d'apprentissage et evaluer le risque de prediction sur l'echantillon test.


In [56]:
Xtrain, Xtest, ytrain, ytest = sk.model_selection.train_test_split(Z,Y,test_size=0.33)

In [57]:
fhat = model.fit(Xtrain, ytrain)
np.mean((fhat.predict(Xtest)-ytest)**2)

9402.561292698803

In [58]:
PhiXtrain, PhiXtest, ytrain, ytest = sk.model_selection.train_test_split(PhiX, Y, test_size=0.33)
fhat2 = model.fit(PhiXtrain, ytrain)
np.mean((fhat2.predict(PhiXtest) - ytest)**2)

87513.2006043595

In [59]:
# Recurer plusieur fois 
cvmodel1 = sk.model_selection.cross_validate(model, Z, Y, scoring="neg_mean_squared_error", cv=5)
cvmodel2 = sk.model_selection.cross_validate(model, PhiX, Y, scoring="neg_mean_squared_error", cv=5)
np.mean(-cvmodel1['test_score'])


10148.391001170392

In [60]:
np.mean(-cvmodel2['test_score'])

9165.3202378132155

# Kernel Regression

In [69]:
from sklearn.kernel_ridge import KernelRidge
Z = pd.get_dummies(X, drop_first=True)
model2 = KernelRidge(kernel = "polynomial", degree = 2, alpha = 30)
cvmodel22 = sk.model_selection.cross_validate(model2, Z, Y, scoring="neg_mean_squared_error", cv = 20)
np.mean(-cvmodel22['test_score'])**(0.5)


62.298117352521238

In [70]:
np.std(Y)

459.18381915633773